In [20]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem

import gzip, cPickle
import copy
import progressbar

In [21]:
## Alkynes
# M = markovnikov, N = anti-markovnikov
# Requires [Si] marker for M/N or carbene reactions

RED = AllChem.ReactionFromSmarts('[C:1]#[C:2].[H][H]>>[C:1][C:2]')

# stereochemistry not regarded
RED_LIND = AllChem.ReactionFromSmarts('[C:1]#[C:2].[H][H]>>[C:1]=[C:2]')
RED_BIRCH = AllChem.ReactionFromSmarts('[C:1]#[C:2]>>[C:1]=[C:2]')

X2 = AllChem.ReactionFromSmarts('[C:1]#[C:2].[Cl,Br:3][Cl,Br:4]>>[C:1]([X:3])([X:3])[C:2]([X:4])([X:4])')

HX = AllChem.ReactionFromSmarts('[Si:1]#[C:2].[Cl,Br,I:3]>>[C:1]([X:3])([X:3])[C:2]')
H2O_M = AllChem.ReactionFromSmarts('[Si:1]#[C:2].[O:3]>>[C:1](=[O:3])[C:2]')
H2O_N = AllChem.ReactionFromSmarts('[Si:1]#[C:2]>>[C:1][C:2](=O)')

OX = AllChem.ReactionFromSmarts('[C:1]#[C:2]>>[C:1](=O)[C:2](=O)')
OX_BASE = AllChem.ReactionFromSmarts('[C:1][C:2]#[C:3][C:4]>>[C:1][C:2](=O)O.[C:4][C:3](=O)O')
OX_BASE_H = AllChem.ReactionFromSmarts('[C:1][C:2]#[C;H1]>>[C:1][C:2](=O)O')

alkyne_MN_reactions_list = [
    ('HX', HX, ['Cl', 'Br', 'I'], ['']),
    
    ('H2O_M', H2O_M, ['O'], ['[Hg+2].[O-]S([O-])(=O)=O.OS(O)(=O)=O']),
    ('H2O_N', H2O_N, [''], ['CC(C)C(C)BC(C)C(C)C.C1CCOC1.OO.[Na+].[OH-]']),
    
]

alkyne_reactions_list = [
    
    ('RED', RED, ['[H][H]'], ['[Pt]', '[Pd]', '[Ni]', '']),
    ('RED_LIND', RED_LIND, ['[H][H]'], ['[Pd].[Ba+2].[O-]S([O-])(=O)=O']),
    ('RED_BIRCH', RED_BIRCH, [''], ['[Na].N']),
    
    ('X2', X2, ['ClCl', 'BrBr'], ['', 'ClCCl']),
    ('OX', OX, [''], ['[K+].[O-][Mn](=O)(=O)=O']),
    ('OX_BASE', OX_BASE, [''], ['[K+].[O-][Mn](=O)(=O)=O.[Na+].[OH-]', 'O=[O+][O-].O']),
    ('OX_BASE_H', OX_BASE_H, [''], ['[K+].[O-][Mn](=O)(=O)=O.[Na+].[OH-]', 'O=[O+][O-].O']),
    
]

In [23]:
def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else ''

In [22]:
with gzip.open('data/subst/alkyne.pkl.gz', 'rb') as f:
    alkyne_list = cPickle.load(f)

length = len(alkyne_list)
print(length)

1862


In [26]:
rxns = []

bar = progressbar.ProgressBar(max_value=length)

# choose alkene
for i, alkyne_smi in enumerate(alkyne_list):
    alkyne = Chem.MolFromSmiles(alkyne_smi)
    alkyne_MN = copy.deepcopy(alkyne)

    # mark [Si]
    C_1, C_2 = alkyne_MN.GetSubstructMatches(Chem.MolFromSmiles('C#C'))[0]
    alkyne_MN.GetAtomWithIdx((C_2 if (alkyne_MN.GetAtomWithIdx(C_1).GetTotalNumHs() \
            > alkyne_MN.GetAtomWithIdx(C_2).GetTotalNumHs()) else C_1)).SetAtomicNum(14);

    # MN reactions
    for reaction in alkyne_MN_reactions_list:
        for reagent in reaction[2]:
            if reagent == '': products = reaction[1].RunReactants((alkyne_MN,))
            else: products = reaction[1].RunReactants((alkyne_MN, Chem.MolFromSmiles(reagent)))
            if len(products) == 0: continue
            product_smi = [Chem.MolToSmiles(product) for product in products[0]]
            for sub_reagent in reaction[3]:
                rxns.append(cano(alkyne_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

    # general reactions
    for reaction in alkyne_reactions_list:
        for reagent in reaction[2]:
            if reagent == '': products = reaction[1].RunReactants((alkyne,))
            else: products = reaction[1].RunReactants((alkyne, Chem.MolFromSmiles(reagent)))
            if len(products) == 0: continue
            product_smi = [Chem.MolToSmiles(product) for product in products[0]]
            for sub_reagent in reaction[3]:
                rxns.append(cano(alkyne_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))
                
    bar.update(i)

 99% (1861 of 1862) |###################### | Elapsed Time: 0:00:08 ETA: 0:00:00

In [27]:
print(len(rxns))

33468


In [28]:
rxns[1:100]

['C#CC(C)C.Br>>CC(C)C(C)(Br)Br',
 'C#CC(C)C.I>>CC(C)C(C)(I)I',
 'C#CC(C)C.O>O=S(=O)(O)O.O=S(=O)([O-])[O-].[Hg+2]>CC(=O)C(C)C',
 'C#CC(C)C>C1CCOC1.CC(C)C(C)BC(C)C(C)C.OO.[Na+].[OH-]>CC(C)CC=O',
 'C#CC(C)C.[H][H]>[Pt]>CCC(C)C',
 'C#CC(C)C.[H][H]>[Pd]>CCC(C)C',
 'C#CC(C)C.[H][H]>[Ni]>CCC(C)C',
 'C#CC(C)C.[H][H]>>CCC(C)C',
 'C#CC(C)C.[H][H]>O=S(=O)([O-])[O-].[Ba+2].[Pd]>C=CC(C)C',
 'C#CC(C)C>N.[Na]>C=CC(C)C',
 'C#CC(C)C.ClCl>>CC(C)C(Cl)(Cl)C(Cl)Cl',
 'C#CC(C)C.ClCl>ClCCl>CC(C)C(Cl)(Cl)C(Cl)Cl',
 'C#CC(C)C.BrBr>>CC(C)C(Br)(Br)C(Br)Br',
 'C#CC(C)C.BrBr>ClCCl>CC(C)C(Br)(Br)C(Br)Br',
 'C#CC(C)C>O=[Mn](=O)(=O)[O-].[K+]>CC(C)C(=O)C=O',
 'C#CC(C)C>O=[Mn](=O)(=O)[O-].[K+].[Na+].[OH-]>CC(C)C(=O)O',
 'C#CC(C)C>O.O=[O+][O-]>CC(C)C(=O)O',
 'C#CCCC.Cl>>CCCC(C)(Cl)Cl',
 'C#CCCC.Br>>CCCC(C)(Br)Br',
 'C#CCCC.I>>CCCC(C)(I)I',
 'C#CCCC.O>O=S(=O)(O)O.O=S(=O)([O-])[O-].[Hg+2]>CCCC(C)=O',
 'C#CCCC>C1CCOC1.CC(C)C(C)BC(C)C(C)C.OO.[Na+].[OH-]>CCCCC=O',
 'C#CCCC.[H][H]>[Pt]>CCCCC',
 'C#CCCC.[H][H]>[Pd]>CCCCC',
 'C